In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.nn.utils.rnn import pad_sequence
from collections import Counter
import re
import underthesea


In [2]:
def lower_tokenize(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    tokens = underthesea.word_tokenize(text)
    # print("Lower",tokens)
    return tokens

In [3]:
def remove_stopword(tokens):
    with open("vietnamese-stopwords.txt","r",encoding="utf-8") as file:
        stopwords = file.readlines()
    stopwords = [word.strip() for word in stopwords]
    final_tokens = [token for token in tokens if token not in stopwords]
    final_text = ' '.join(final_tokens)
    # print("Stopword",final_text)
    return final_text


In [ ]:
# Tải dữ liệu
data = pd.read_csv('../Crawl/Data1.csv')
data = data.dropna(subset=['Nội dung', 'Danh mục'])
data['tokens'] = data['Nội dung'].apply(lower_tokenize)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4314 entries, 0 to 4313
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Danh mục  4314 non-null   object
 1   Nội dung  4314 non-null   object
 2   tokens    4314 non-null   object
dtypes: object(3)
memory usage: 101.2+ KB


In [ ]:
data['Danh mục'].value_counts()

Danh mục
Xe          680
Giới trẻ    562
Văn hóa     548
Thế giới    510
Thời sự     380
Kinh tế     295
Sức khỏe    283
Giáo dục    271
Đời sống    250
Thể thao    250
Giải trí    147
Du lịch     138
Name: count, dtype: int64

In [ ]:
df = data.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4314 entries, 0 to 4313
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Danh mục  4314 non-null   object
 1   Nội dung  4314 non-null   object
 2   tokens    4314 non-null   object
dtypes: object(3)
memory usage: 101.2+ KB


In [ ]:
# Mã hóa nhãn
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['Danh mục'])

In [ ]:
# Tạo vocab
all_tokens = [token for tokens in df['tokens'] for token in tokens]
vocab = {word: idx + 2 for idx, (word, _) in enumerate(Counter(all_tokens).items())}
vocab['<PAD>'] = 0
vocab['<UNK>'] = 1


In [ ]:
# Chuyển tokens thành ids
def encode(tokens):
    return [vocab.get(token, 1) for token in tokens]

data['input_ids'] = data['tokens'].apply(encode)

In [ ]:
# Dataset PyTorch
class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return torch.tensor(self.X[idx]), torch.tensor(self.y[idx])

def collate_fn(batch):
    inputs, labels = zip(*batch)
    inputs = pad_sequence(inputs, batch_first=True, padding_value=0)
    labels = torch.tensor(labels)
    return inputs, labels

In [ ]:
# Tách tập train/test
X_train, X_test, y_train, y_test = train_test_split(data['input_ids'].tolist(), y.tolist(), test_size=0.4, random_state=42)
train_dataset = TextDataset(X_train, y_train)
test_dataset = TextDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)


In [ ]:

# Mô hình BiLSTM với PyTorch
class BiLSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes):
        super(BiLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, bidirectional=True, batch_first=True)
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(hidden_dim * 2, num_classes)
    def forward(self, x):
        x = self.embedding(x)
        lstm_out, _ = self.lstm(x)
        out = lstm_out[:, -1, :]
        out = self.dropout(out)
        return self.fc(out)

In [ ]:
# Khởi tạo mô hình
vocab_size = len(vocab)
embed_dim = 128
hidden_dim = 64
num_classes = len(label_encoder.classes_)
model = BiLSTM(vocab_size, embed_dim, hidden_dim, num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BiLSTM(
  (embedding): Embedding(100286, 128, padding_idx=0)
  (lstm): LSTM(128, 64, batch_first=True, bidirectional=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=128, out_features=12, bias=True)
)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 10

for epoch in range(epochs):
    model.train()
    all_preds = []
    all_labels = []
    total_loss = 0.0

    for inputs, labels in train_loader:
        inputs = inputs.long()  # Ép kiểu sang LongTensor
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Lưu lại dự đoán và nhãn thật để tính accuracy
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    # Tính accuracy sau mỗi epoch
    acc = accuracy_score(all_labels, all_preds)
    avg_loss = total_loss / len(train_loader)

    print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f} - Accuracy: {acc:.4f}")



Epoch 1/10 - Loss: 2.3613 - Accuracy: 0.1820
Epoch 2/10 - Loss: 2.3586 - Accuracy: 0.1824
Epoch 3/10 - Loss: 2.3552 - Accuracy: 0.1832
Epoch 4/10 - Loss: 2.3479 - Accuracy: 0.1847
Epoch 5/10 - Loss: 2.3410 - Accuracy: 0.1889
Epoch 6/10 - Loss: 2.3387 - Accuracy: 0.1886
Epoch 7/10 - Loss: 2.3402 - Accuracy: 0.1874
Epoch 8/10 - Loss: 2.3382 - Accuracy: 0.1855
Epoch 9/10 - Loss: 2.3548 - Accuracy: 0.1882
Epoch 10/10 - Loss: 2.3285 - Accuracy: 0.1917


In [ ]:
# Tính precision, recall, f1-score
model.eval()
y_pred_labels = []
y_true_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        y_pred_labels.extend(preds)
        y_true_labels.extend(labels.numpy())

precision = precision_score(y_true_labels, y_pred_labels, average='weighted', zero_division=0)
recall = recall_score(y_true_labels, y_pred_labels, average='weighted', zero_division=0)
f1 = f1_score(y_true_labels, y_pred_labels, average='weighted', zero_division=0)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

Precision: 0.1145
Recall: 0.1547
F1-score: 0.0516


In [ ]:
# 1. Chuẩn bị mẫu dữ liệu cần kiểm thử
sample_texts = [
    "Xe máy CRB1000R vừa ra mắt mẫu mới",
    "Việt Nam ra sân thắng Lào với tỉ số 10-0",
    "Bạo lực học đường là vấn đề được quan tâm nhất hiện nay",
    
]

# 2. Tiền xử lý: token hóa + loại bỏ stopword
processed_samples = []
for text in sample_texts:
    tokens = lower_tokenize(text)
    filtered_text = remove_stopword(tokens)
    final_tokens = filtered_text.split()  # Chuyển lại thành list từ string sau khi remove_stopword
    processed_samples.append(final_tokens)

# 3. Mã hóa bằng vocab đã tạo
encoded_samples = [torch.tensor(encode(tokens)) for tokens in processed_samples]

# 4. Padding cho input
padded_samples = pad_sequence(encoded_samples, batch_first=True, padding_value=0).to(device)

# 5. Dự đoán
model.eval()
with torch.no_grad():
    outputs = model(padded_samples)
    predictions = torch.argmax(outputs, dim=1)

import joblib
tfidf_vectorizer, MultiNB, label_encoder = joblib.load("model.joblib")
predicted_labels = label_encoder.inverse_transform(predictions.cpu().numpy())

# 7. Hiển thị kết quả
for text, label in zip(sample_texts, predicted_labels):
    print("Nội dung:", text)
    print("→ Dự đoán danh mục:", label)
    print("-" * 100)


Nội dung: Xe máy CRB1000R vừa ra mắt mẫu mới
→ Dự đoán danh mục: Xe
----------------------------------------------------------------------------------------------------
Nội dung: Việt Nam ra sân thắng Lào với tỉ số 10-0
→ Dự đoán danh mục: Xe
----------------------------------------------------------------------------------------------------
Nội dung: Bạo lực học đường là vấn đề được quan tâm nhất hiện nay
→ Dự đoán danh mục: Xe
----------------------------------------------------------------------------------------------------
